Let's perform EDA and basic training using data from the [IEEE-CIS Fraud Detection](https://www.kaggle.com/competitions/ieee-fraud-detection/) Kaggle competition.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dataprep.eda import create_report

In [ ]:
train_df = pd.read_csv("./data/train_transaction.csv")
test_df = pd.read_csv("./data/test_transaction.csv")

train_id = pd.read_csv("./data/train_identity.csv")
test_id = pd.read_csv("./data/test_identity.csv")

train_df = pd.merge(train_df, train_id, on="TransactionID", how="left")
test_df = pd.merge(test_df, test_id, on="TransactionID", how="left")

print(train_df.shape)
train_df.head()

In [ ]:
is_na = train_df.isna().sum() / len(train_df)

print(f"{(is_na > 0).sum()} out of {len(is_na)} columns have missing values")
plt.figure(figsize=(15, 4))
ax = plt.bar(np.arange(len(is_na)), is_na.values)

There are variables with tons of missing data.